In [ ]:
import pandas as pd
import seaborn as sns
import os

In [ ]:
HC_path = '/Users/jk1/temp/stroke_resilience/output/stroke_resilience/HC/basic_graph_metrics.csv'
ST01_path = '/Users/jk1/temp/stroke_resilience/output/stroke_resilience/ST01/basic_graph_metrics.csv'
ST02_path = '/Users/jk1/temp/stroke_resilience/output/stroke_resilience/ST02/basic_graph_metrics.csv'
ST03_path = '/Users/jk1/temp/stroke_resilience/output/stroke_resilience/ST03/basic_graph_metrics.csv'

output_save_path = '/Users/jk1/temp/stroke_resilience/output/figures'

In [ ]:

HC_metrics = pd.read_csv(HC_path)
HC_metrics['timepoint_group'] = 'HC'

ST01_metrics = pd.read_csv(ST01_path)
ST01_metrics['timepoint_group'] = 'ST01'

ST02_metrics = pd.read_csv(ST02_path)
ST02_metrics['timepoint_group'] = 'ST02'

ST03_metrics = pd.read_csv(ST03_path)
ST03_metrics['timepoint_group'] = 'ST03'

all_groups_metrics = pd.concat([HC_metrics, ST01_metrics, ST02_metrics, ST03_metrics], ignore_index=True)

In [ ]:
# Drop threshold < 0.3
all_groups_metrics = all_groups_metrics[all_groups_metrics.threshold != 'top0']
all_groups_metrics = all_groups_metrics[all_groups_metrics.threshold != 'top10']
all_groups_metrics = all_groups_metrics[all_groups_metrics.threshold != 'top20']


In [ ]:
all_groups_metrics = all_groups_metrics.drop('subject', axis=1)

In [ ]:
mean_metrics = all_groups_metrics.groupby(
    ['timepoint_group', 'threshold']
).mean()
mean_metrics

In [ ]:
# transform back into normal df
mean_metrics = mean_metrics.reset_index()

In [ ]:
# sort thresholds
mean_metrics['sort'] = mean_metrics['threshold'].str.extract('(\d+)', expand=False).astype(int)
mean_metrics = mean_metrics.sort_values('sort', ascending=True)
# sort groups
mean_metrics = mean_metrics.sort_values(['timepoint_group', 'sort'], ascending=True)
mean_metrics = mean_metrics.drop('sort', axis=1)

In [ ]:
mean_metrics.to_csv(os.path.join(output_save_path, 'mean_basic_graph_metrics.csv'))

In [ ]:
def create_and_save_plot(dependant_variable:str):
    g = sns.relplot(data=mean_metrics, x='timepoint_group', y=dependant_variable, hue='threshold', style='threshold',
                alpha=.7, kind="line", palette='flare')

    g.fig.suptitle(dependant_variable, y=1.03)
    g.savefig(os.path.join(output_save_path, dependant_variable + ".png"))

In [ ]:
create_and_save_plot('n_nodes')
create_and_save_plot('n_edges')
create_and_save_plot('mean_degree')
